OrderBook Demo
-------------

Examples of placing orders

In [1]:
#Mosie Schrem
import pandas as pd
import numpy as np
import os
import math
import sys
from matplotlib import pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
sys.path.insert(0, os.path.join(os.getcwd(), 'code'))

import OrderBook as ob
import OrderUtil as ou

Examples of creating and processing orders:
--------

In [2]:
#load in intc OrderBook data
book = ob.OrderBook(message_filename='INTC_message.csv', orderbook_filename='INTC_OrderBook.csv')

In [3]:
limit_book = book.limit_order_book()
limit_book['mid'] = limit_book['ap1'] - limit_book['bp1']
limit_book = limit_book.loc[(limit_book['ap1'] - limit_book['bp1']).shift(1) >= 
                            limit_book['ap1'] - limit_book['bp1']]
limit_book = limit_book.loc[limit_book['mid'].shift(1) != limit_book['mid']]
limit_book['label'] = [1 if limit_book['mid'].shift(1).values[i] < limit_book['mid'].values[i] else -1
                      for i in range(len(limit_book))]

In [4]:
print(len(limit_book))

2956


Create an ask order at level 2, at time 38000
-------
We reevaluate order status at every BookUpdate 

In [5]:
order = ou.BookUpdatesOrder(orderbook=book, numupdates=1, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'queue_position': 16646, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [6]:
#process 100 updates to orderbook
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38014.459199206, 'is_buy': False, 'queue_position': 16446, 'orderstate': 'open', 'level': 2, 'current_index': 132039}


In [7]:
#process another 300 updates
for i in range(300):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38015.006473335, 'is_buy': False, 'queue_position': 16346, 'orderstate': 'open', 'level': 2, 'current_index': 132339}


In [8]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
    
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'queue_position': 0, 'orderstate': 'executed', 'level': 1, 'end_index': 134180}


We can also create orders by timestep of fixed size
--------

Create an ask order at level 2, at time 38000, reevaluating order status every 0.5 seconds

In [9]:
order = ou.TimeOrder(orderbook=book, delta_t=0.5, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'queue_position': 16646, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [10]:
#process 50 seconds of data
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38049.987429576, 'is_buy': False, 'queue_position': 13946, 'orderstate': 'open', 'level': 3, 'current_index': 132820}


In [11]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'queue_position': 0, 'orderstate': 'executed', 'level': 1, 'end_index': 134180}
